<a href="https://colab.research.google.com/github/ducpc99/MLPs_CIFAR_10/blob/main/MLPs_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

## Data Preparation

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Thiết lập tham số
batch_size = 128
weight_decay = 1e-4
dropout_rate = 0.5

# Tạo DataLoader với batch_size mới
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


## Define MLPs

In [ ]:
class MLP(nn.Module):
    def __init__(self, hidden_layers=3, hidden_units=1024, dropout_rate=0.5):
        super(MLP, self).__init__()
        self.input_layer = nn.Linear(32*32*3, hidden_units)
        self.hidden_layers = nn.ModuleList([nn.Linear(hidden_units, hidden_units) for _ in range(hidden_layers)])
        self.output_layer = nn.Linear(hidden_units, 10)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = x.view(-1, 32*32*3)
        x = self.activation(self.input_layer(x))
        x = self.dropout(x)
        for layer in self.hidden_layers:
            x = self.activation(layer(x))
            x = self.dropout(x)
        x = self.output_layer(x)
        return x

## Model Training and Evaluation

In [ ]:
def train_model(model, train_loader, criterion, optimizer, scheduler, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Cập nhật scheduler
        scheduler.step()

        # Tính loss trung bình và in ra
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

        # Đánh giá độ chính xác trên tập kiểm tra mỗi 5 epochs
        if (epoch + 1) % 5 == 0:
            accuracy = evaluate_model(model, test_loader)
            print(f"Validation Accuracy after Epoch {epoch+1}: {accuracy:.2f}%\n")

def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total * 100  # Tính độ chính xác theo %
    return accuracy

## Device Setup

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model = MLP(hidden_layers=3, hidden_units=1024, dropout_rate=dropout_rate).to(device)

if device.type == "cuda":
    print(torch.cuda.get_device_name(0))

Using device: cuda
Tesla T4


## Model Initialization and Training

In [ ]:
# Khởi tạo model với dropout
model = MLP(hidden_layers=2, hidden_units=512, dropout_rate=dropout_rate).to(device)
epochs = 50  # Tăng số lượng epoch
learning_rate = 0.0001  # Giảm learning rate để mô hình học chậm hơn và ổn định hơn
gamma = 0.5
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=gamma)
# Khởi tạo optimizer với weight decay
criterion = nn.CrossEntropyLoss()
train_model(model, train_loader, criterion, optimizer, scheduler, epochs)
evaluate_model(model, test_loader)


Epoch [1/50], Loss: 2.0594
Epoch [2/50], Loss: 1.8894
Epoch [3/50], Loss: 1.8198
Epoch [4/50], Loss: 1.7789
Epoch [5/50], Loss: 1.7484
Validation Accuracy after Epoch 5: 42.02%

Epoch [6/50], Loss: 1.5963
Epoch [7/50], Loss: 1.5440
Epoch [8/50], Loss: 1.5038
Epoch [9/50], Loss: 1.4749
Epoch [10/50], Loss: 1.4456
Validation Accuracy after Epoch 10: 48.29%

Epoch [11/50], Loss: 1.4240
Epoch [12/50], Loss: 1.4012
Epoch [13/50], Loss: 1.3861
Epoch [14/50], Loss: 1.3681
Epoch [15/50], Loss: 1.3536
Validation Accuracy after Epoch 15: 51.93%

Epoch [16/50], Loss: 1.3400
Epoch [17/50], Loss: 1.3280
Epoch [18/50], Loss: 1.3119
Epoch [19/50], Loss: 1.2984
Epoch [20/50], Loss: 1.2918
Validation Accuracy after Epoch 20: 52.90%

Epoch [21/50], Loss: 1.2570
Epoch [22/50], Loss: 1.2475
Epoch [23/50], Loss: 1.2408
Epoch [24/50], Loss: 1.2300
Epoch [25/50], Loss: 1.2290
Validation Accuracy after Epoch 25: 54.46%

Epoch [26/50], Loss: 1.2186
Epoch [27/50], Loss: 1.2140
Epoch [28/50], Loss: 1.2091
Epoch 

57.15